# Adversarial Training
In recent years, people have found that even small perturbations can lead models make false prediction, which means the robustness of model is not good. How to improve the robustness of the model becomes a problem, Adversarial training is a method to improve the robustness of models. 

In this notebook, we will show you the model's not robust under the attack produced by textflint. And how adversarial training can help improve the robustness of the same model.

## Prepare Dataset

First, you need to prepare data sets for training and testing. Here we use 5000 pieces of data for training and 500 pieces of data for testing, all data are from the IMDB data set

In [1]:
from textflint.input import Dataset
# load your data here, we use 5000 train data and 500 test data collected from IMDB dataset
train_dataset_path = 'dataset/train.json'
test_dataset_path = 'dataset/test.json'

# Dataset is defined to use TextFlint  more efficiently
train_data = Dataset(task='SA')
train_data.load_json(train_dataset_path)

test_data = Dataset(task='SA')
test_data.load_json(test_dataset_path)

TextFlint: ******Start load!******
100%|██████████| 5000/5000 [00:11<00:00, 417.87it/s]
TextFlint: 5000 in total, 5000 were loaded successful.
TextFlint: ******Finish load!******
TextFlint: ******Start load!******
100%|██████████| 500/500 [00:00<00:00, 525.47it/s]
TextFlint: 500 in total, 500 were loaded successful.
TextFlint: ******Finish load!******


## Prepare model and training process

Here we take TextCNN as target model, you should define your target model with pytorch or tensorflow and provide training process. 

In order to make the model more effectively used by TextFlint, you need to wrapper your model with FlintModel, some functions such as `__call__` and `encode` need to be implemented. More details are available in https://github.com/textflint/textflint/blob/master/textflint/input/model/flint_model/flint_model.py

In [2]:
import torch
import os
import numpy as np
from tqdm.notebook import tqdm
from torch.nn import functional as F
from textflint.common.utils import logger
from textflint.input.model import FlintModel
from textflint.input.model.test_model.textcnn_torch_model import TextCNNTorchModel
from textflint.input.model.tokenizers.glove_tokenizer import  GloveTokenizer
from textflint.input.model.test_model.glove_embedding import  GloveEmbedding
from textflint.input.model.test_model.model_helper import *

class CNNFlint(FlintModel):
    r"""
    Model wrapper for TextCnn implemented by pytorch.

    """
    def __init__(self, model, tokenizer, label2id):
        super().__init__(
            model=model,
            task='SA',
            tokenizer=tokenizer
        )
        self.label2id = label2id

    def __call__(self, batch_texts):
        r"""
        Tokenize text, convert tokens to id and run the model.

        :param batch_texts: (batch_size,) batch text input
        :return: numpy.array()

        """
        model_device = next(self.model.parameters()).device
        inputs_ids = [self.encode(batch_text) for batch_text in batch_texts]
        ids = torch.tensor(inputs_ids).to(model_device)

        return self.model(ids).detach().cpu().numpy()

    def encode(self, inputs):
        r"""
        Tokenize inputs and convert it to ids.

        :param inputs: model original input
        :return: list of inputs ids

        """
        return self.tokenizer.encode(inputs)

    def unzip_samples(self, data_samples):
        r"""
        Unzip sample to input texts and labels.

        :param list[Sample] data_samples: list of Samples
        :return: (inputs_text), labels.

        """
        x = []
        y = []

        for sample in data_samples:
            x.append(sample['x'])
            y.append(self.label2id[sample['y']])

        return [x], y


def train(training_data, args):
    '''
    Train process for TextCNN, create a TextCNN model and train it with training_data and args
    '''

    model = TextCNNTorchModel(init_embedding=args['embedding'].embedding)
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
    epoch_bar = tqdm(range(1, args['epoch'] + 1))
    for epoch in epoch_bar:
        total_loss = 0
        steps = 0
        corrects = 0
        for inputs, labels in train_iter(training_data, args['batch_size'], args['tokenizer'], args['label2id']):
            inputs, labels = torch.from_numpy(inputs).to(args['device']), torch.from_numpy(labels).to(args['device'])
            optimizer.zero_grad()
            logits = model(inputs)

            loss = F.cross_entropy(logits, labels)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
            
            result = torch.max(logits, 1)[1].view(labels.size())
            steps += 1
            corrects += (result.data == labels.data).sum()
        accuracy = corrects * 100.0 / (args['batch_size']*steps)
#         epoch_bar.set_description('epoch {}'.format(epoch))
        epoch_bar.set_description('loss({:.2f}) train acc({:.2f})'.format(total_loss, accuracy))
#     print(f"epoch {epoch}\t loss {total_loss} train acc {accuracy}")

    return model


# Essential parts(embedding, tokenizer) and args for training TextCNN
glove_embedding = GloveEmbedding()
tokenizer = GloveTokenizer(
                    word_id_map=glove_embedding.word2id,
                    unk_token_id=glove_embedding.oovid,
                    pad_token_id=glove_embedding.padid,
                    max_length=512
                )
label2id = {"0": 0, "1": 1}
train_args = {'lr': 0.0001, 'epoch': 30, 'batch_size': 64, 'device': 'cuda:0', 'max_length': 512, 'label2id':label2id, 'tokenizer':tokenizer, 'embedding':glove_embedding}
cnn_model = train(training_data=train_data.dump(), args=train_args)

# Wrapper trained model with CNNFlint
cnn_flint = CNNFlint(model=cnn_model, tokenizer=tokenizer, label2id=label2id)


/home/xin_zhou/anaconda3/envs/demo/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


## Transform data

We use TextFint to generate new testset to evaluate the robustness of model

In [3]:
from textflint.engine import Engine
from textflint.adapter import auto_config
from textflint.input.config import Config
from textflint.common.settings import UNMATCH_UT_TRANSFORMATIONS, \
    TASK_TRANSFORMATIONS

# Config for TextFlint to transform data, see more details in https://textflint.readthedocs.io/en/latest/
config = auto_config(task='SA')
config.trans_methods = ['DoubleDenial', 'Ocr']
config.sub_methods = []
engine = Engine()
# Generate adversarial train data, saved in ./train_results dir
config.out_dir = './train_results'
engine.run(data_input=train_dataset_path, config=config)

# Generate adversarial test data, saved in ./test_results dir
config.out_dir = './test_results'
engine.run(data_input=test_dataset_path, config=config)


TextFlint: ******Start load!******
100%|██████████| 5000/5000 [00:06<00:00, 744.43it/s]
TextFlint: 5000 in total, 5000 were loaded successful.
TextFlint: ******Finish load!******
TextFlint: ******Start DoubleDenial!******
100%|██████████| 5000/5000 [01:51<00:00, 45.03it/s]
TextFlint: DoubleDenial, original 5000 samples, transform 4606 samples!
TextFlint: Save samples to ./train_results/ori_DoubleDenial_4606.json!
TextFlint: Save samples to ./train_results/trans_DoubleDenial_4606.json!
TextFlint: ******Finish DoubleDenial!******
TextFlint: ******Start Ocr!******
100%|██████████| 5000/5000 [00:33<00:00, 150.99it/s]
TextFlint: Ocr, original 5000 samples, transform 5000 samples!
TextFlint: Save samples to ./train_results/ori_Ocr_5000.json!
TextFlint: Save samples to ./train_results/trans_Ocr_5000.json!
TextFlint: ******Finish Ocr!******
TextFlint: ******Start load!******
100%|██████████| 500/500 [00:00<00:00, 772.28it/s]
TextFlint: 500 in total, 500 were loaded successful.
TextFlint: *****

## Evaluate robustness of model

Next step we using transformed test data to evaluate the performance of original TextCNN model. You can see that the performance of model is reduced on the transformed sample. This proves that the robustness of TextCNN is not very good. It's a very common phenomenon, not just in the TextCNN model. But adversarial training is useful to alleviate this problem.

In [4]:
import re
def find_target_data(trans_name, output_dir, prefix='trans'):
    '''
    Find target data generated by TextFlint in output_dir
    '''
    file_list = os.listdir(output_dir)
    for file in file_list:
        if re.match('{}_{}_\d+.json'.format(prefix, trans_name), file):
            return '{}/{}'.format(output_dir, file)
    return None

def compare_evaluate(flintmodel, trans_name, output_dir='./test_results'):
    '''
    Evaluate flintmodel in original test data and transformed test data
    '''
    ori_test_data = Dataset(task='SA')
    ori_test_data.load_json(find_target_data(trans_name=trans_name, output_dir=output_dir, prefix='ori'))
    trans_test_data = Dataset(task='SA')
    trans_test_data.load_json(find_target_data(trans_name=trans_name, output_dir=output_dir, prefix='trans'))

    flintmodel.model.eval()
    ori_eval_result = flintmodel.evaluate(ori_test_data.dump(), prefix='{}_ori_'.format(trans_name))
    trans_eval_result = flintmodel.evaluate(trans_test_data.dump(), prefix='{}_trans_'.format(trans_name))
    return ori_eval_result, trans_eval_result

ori_model_results = []

# Evaluate cnn_flint in original test data and transformed test data.
for trans_method in config.trans_methods:
#     print('Evaluate the performence of TextCNN {}'.format(trans_method))
    cnn_results = compare_evaluate(cnn_flint, trans_method)
    ori_model_results.append(cnn_results)
#     print('Adversarial training for TextCNN {}'.format(trans_method))

ori_model_results

TextFlint: ******Start load!******
100%|██████████| 461/461 [00:00<00:00, 742.60it/s]
TextFlint: 461 in total, 461 were loaded successful.
TextFlint: ******Finish load!******
TextFlint: ******Start load!******
100%|██████████| 461/461 [00:00<00:00, 702.36it/s]
TextFlint: 461 in total, 461 were loaded successful.
TextFlint: ******Finish load!******
TextFlint: ******Start load!******
100%|██████████| 500/500 [00:00<00:00, 699.62it/s]
TextFlint: 500 in total, 500 were loaded successful.
TextFlint: ******Finish load!******
TextFlint: ******Start load!******
100%|██████████| 500/500 [00:00<00:00, 563.69it/s]
TextFlint: 500 in total, 500 were loaded successful.
TextFlint: ******Finish load!******


[({'DoubleDenial_ori_accuracy': 0.8459869848156182},
  {'DoubleDenial_trans_accuracy': 0.665943600867679}),
 ({'Ocr_ori_accuracy': 0.844}, {'Ocr_trans_accuracy': 0.84})]

## Adversarial training

To enhance model's robustness, we add the adversarial sample to original dataset and re-train the model. (There are lots of ways to do adversarial training, we choose the most common one). We found that the performance of the model with adversarial training improved significantly comparing to the same model without adversarial training as we expected.

In [5]:
import re

def adv_train(trans_name, output_dir='./train_results'):
    '''
    Adversarial training for TextCNN and target transformation
    '''
    # Mix up original training data and adversarial sample
    mix_train_data = Dataset(task='SA')
    mix_train_data.load_json(find_target_data(trans_name=trans_name, output_dir=output_dir))
    mix_train_data.load_json(train_dataset_path)
    
    # retrain TextCNN with mixed data and evaluate it in original test data and transformed test data
    adv_cnn_model = train(training_data=mix_train_data.dump(), args=train_args)
    adv_cnn_flint = CNNFlint(model=adv_cnn_model, tokenizer=tokenizer, label2id=label2id)
    return adv_cnn_flint

adv_model_results = []

# adversarial training for cnn_flint, and evaluate new model in original test data and transformed data.
for trans_method in config.trans_methods:
    adv_cnn_flint = adv_train(trans_method)
    adv_cnn_result = compare_evaluate(adv_cnn_flint, trans_method)
    adv_model_results.append(adv_cnn_result)

# Compare the performance of original model and adversarial training model
print('-----------------Original TextCNN model results-----------------')
print(ori_model_results)
print('-----------------TextCNN with adversarial training results-----------------')
print(adv_model_results)

TextFlint: ******Start load!******
100%|██████████| 4606/4606 [00:07<00:00, 621.87it/s]
TextFlint: 4606 in total, 4606 were loaded successful.
TextFlint: ******Finish load!******
TextFlint: ******Start load!******
100%|██████████| 5000/5000 [00:06<00:00, 762.59it/s]
TextFlint: 5000 in total, 5000 were loaded successful.
TextFlint: ******Finish load!******


TextFlint: ******Start load!******
 14%|█▎        | 63/461 [00:00<00:00, 620.68it/s]

100%|██████████| 461/461 [00:00<00:00, 741.02it/s]
TextFlint: 461 in total, 461 were loaded successful.
TextFlint: ******Finish load!******
TextFlint: ******Start load!******
100%|██████████| 461/461 [00:00<00:00, 765.22it/s]
TextFlint: 461 in total, 461 were loaded successful.
TextFlint: ******Finish load!******
TextFlint: ******Start load!******
100%|██████████| 5000/5000 [00:07<00:00, 630.18it/s]
TextFlint: 5000 in total, 5000 were loaded successful.
TextFlint: ******Finish load!******
TextFlint: ******Start load!******
100%|██████████| 5000/5000 [00:06<00:00, 761.84it/s]
TextFlint: 5000 in total, 5000 were loaded successful.
TextFlint: ******Finish load!******


TextFlint: ******Start load!******
 15%|█▌        | 75/500 [00:00<00:00, 749.95it/s]

100%|██████████| 500/500 [00:00<00:00, 767.24it/s]
TextFlint: 500 in total, 500 were loaded successful.
TextFlint: ******Finish load!******
TextFlint: ******Start load!******
100%|██████████| 500/500 [00:00<00:00, 740.54it/s]
TextFlint: 500 in total, 500 were loaded successful.
TextFlint: ******Finish load!******


-----------------Original TextCNN model results-----------------
[({'DoubleDenial_ori_accuracy': 0.8459869848156182}, {'DoubleDenial_trans_accuracy': 0.665943600867679}), ({'Ocr_ori_accuracy': 0.844}, {'Ocr_trans_accuracy': 0.84})]
-----------------TextCNN with adversarial training results-----------------
[({'DoubleDenial_ori_accuracy': 0.8568329718004338}, {'DoubleDenial_trans_accuracy': 0.8459869848156182}), ({'Ocr_ori_accuracy': 0.866}, {'Ocr_trans_accuracy': 0.862})]
